In [ ]:
import os
import torch
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import numpy as np

In [ ]:
def process_tiff(tiff_path, shapefile_path, grid_size=10):
    # Read the shapefile
    california = gpd.read_file(shapefile_path)

    # Read the TIFF file
    with rasterio.open(tiff_path) as src:
        # Crop the TIFF to the California border
        california_bounds = california.geometry.bounds.iloc[0]
        california_geom = [json.loads(california.geometry.to_json())['features'][0]['geometry']]
        out_image, out_transform = mask(src, california_geom, crop=True)

        # Get the width and height of the cropped image
        height, width = out_image.shape[1], out_image.shape[2]

        # Create a list to store the tensors
        tensors = []

        # Create the grid and convert each cell to a tensor
        for i in range(0, height, grid_size):
            for j in range(0, width, grid_size):
                if i + grid_size <= height and j + grid_size <= width:
                    cell = out_image[:, i:i+grid_size, j:j+grid_size]
                    tensors.append(torch.tensor(cell, dtype=torch.float32))

        # Stack all tensors into a single tensor
        final_tensor = torch.stack(tensors)

        return final_tensor

In [ ]:
def save_tensors(tensors, output_path):
    torch.save(tensors, output_path)

if __name__ == "__main__":
    tiff_path = "path/to/your/tiff/file.tif"
    shapefile_path = "EE ML PFAS/Raw_Data/Mapping/California_Shapefile/California.shp"
    output_path = "path/to/save/tensors.pth"

    tensors = process_tiff(tiff_path, shapefile_path)
    save_tensors(tensors, output_path)